In [ ]:
import polars as pl

In [ ]:
# set round off
round_off = 3

# get ohlc dataFrame

ohlc1 = pl.read_csv("ada-usdt-5m.csv")
ohlc = ohlc1[5:]
ohlc = ohlc.drop(["coin_b_high", "coin_b_low", "coin_b_open", "coin_b_close"])
ohlc.head(5)

In [ ]:
# calculate percent change for coin_a = IAG

ohlc = ohlc.with_columns(pl.col("coin_a_open").pct_change().alias("percent_change"))
# ohlc.head(5)
ohlc

In [ ]:
# calculate buy order for different order levels
order_levels: int = 10

for order_level in range(order_levels, 0, -1):
    ratio: float = 0.005
    ohlc = ohlc.with_columns(
        ((pl.col("coin_a_open") * (1 - (order_level*ratio))).round(round_off)).alias(
            f"buy_level_{order_level}"
        )
    )

ohlc.head(5)

In [ ]:
# calculate sell order for different order levels

for order_level in range(1, order_levels + 1):
    ratio: float = 0.005
    ohlc = ohlc.with_columns(
        ((pl.col("coin_a_open") * (1 + (order_level*ratio))).round(round_off)).alias(
            f"sell_level_{order_level}"
        )
    )

ohlc.head(5)

In [ ]:
# calculate buy hedge for different order levels
spread: float = 0.002

for order_level in range(1, order_levels + 1):
    ohlc = ohlc.with_columns(
        ((pl.col(f"sell_level_{order_level}") * (1 - spread)).round(round_off)).alias(
            f"hedge_buy_{order_level}"
        )
    )

ohlc.head(5)

In [ ]:
# calculate sell hedge for different order levels

for order_level in range(1, order_levels + 1):
    ohlc = ohlc.with_columns(
        ((pl.col(f"buy_level_{order_level}") * (1 + spread)).round(round_off)).alias(
            f"hedge_sell_{order_level}"
        )
    )

ohlc.head(5)

In [ ]:
#############################################Analysis#############################################################

In [ ]:
# function to calculate orders executed

from typing import Tuple


def check_orders_executed(
    coin_a_low: float, coin_a_high: float, order: float, level: int, order_type: str
) -> int:
    # check a particular order level of buy and sell
    if coin_a_low < order < coin_a_high:
        print(
            f"order executed: {order_type}_{level} --> low: {coin_a_low} < order: {order} < high: {coin_a_high}, "
        )
        return 1  # order is executed
    else:
        print(
            f"order not executed: {order_type}_{level} --> low: {coin_a_low} < order: {order} < high: {coin_a_high}"
        )
        return 0  # order is not executed


def get_orders_executed(i: int, ohlc: pl.DataFrame) -> Tuple[int, int]:
    # get coin_a_low, coin_a_high
    coin_a_low: float = pl.Series(
        "coin_a_low", ohlc.select(pl.col("coin_a_low")).to_series()
    ).to_list()[i]
    coin_a_high: float = pl.Series(
        "coin_a_high", ohlc.select(pl.col("coin_a_high")).to_series()
    ).to_list()[i]

    # get buy and sell orders executed for each order level
    buy_orders_executed: int = 0
    sell_orders_executed: int = 0

    for order_level in range(1, order_levels + 1):
        buy_order_level: float = pl.Series(
            f"buy_level_{order_level}",
            ohlc.select(pl.col(f"buy_level_{order_level}")).to_series(),
        ).to_list()[i]
        sell_order_level: float = pl.Series(
            f"sell_level_{order_level}",
            ohlc.select(pl.col(f"sell_level_{order_level}")).to_series(),
        ).to_list()[i]

        # check buy order for each order level
        buy_orders_executed += check_orders_executed(
            coin_a_low, coin_a_high, buy_order_level, order_level, "buy"
        )

        # check sell order for each order level
        sell_orders_executed += check_orders_executed(
            coin_a_low, coin_a_high, sell_order_level, order_level, "sell"
        )

    return buy_orders_executed, sell_orders_executed

In [ ]:
# calculate orders executed

buy_order_executed: list[int] = []
sell_order_executed: list[int] = []

for i in range(ohlc.shape[0]):
    print(
        f"--------------------------------------{i}th row in ohlc dataframe-----------------------------------------"
    )
    buy_orders, sell_orders = get_orders_executed(i, ohlc)
    buy_order_executed.append(buy_orders)
    sell_order_executed.append(sell_orders)

ohlc = ohlc.insert_column(
    column=pl.Series("buy_order_executed", buy_order_executed), index=ohlc.shape[1]
)

ohlc = ohlc.insert_column(
    column=pl.Series("sell_order_executed", sell_order_executed), index=ohlc.shape[1]
)

ohlc.head(5)

In [ ]:
# function to calculate tokens on every order executed

def calculate_tokens(initial_tokens: int, order: float, hedge: float, order_type: str ) -> float:
    if order_type == 'buy':     
        # new ada tokens value after buy order is executed with hedges
        ada_tokens: float = (initial_tokens / order) * hedge
        ada_token_gains: float = ada_tokens - initial_tokens
        return ada_token_gains
        
    else:
        # new actual tokens value after sell order is executed with hedges
        actual_tokens: float = (initial_tokens * order) / hedge
        actual_token_gains: float = actual_tokens - initial_tokens
        return actual_token_gains

In [ ]:
# function to calculate profits after every order executed

def calculate_profits(i: int, order_type: str, ohlc: pl.DataFrame, initial_tokens: int) -> Tuple[float, float, float]:
    # list of profits
    profits_data: list[float] = []
    base_token_gains: list[float] = []
    target_token_gains: list[float] = []

    # get coin_a_low, coin_a_high, coin_a_close
    coin_a_low: float = pl.Series(
        "coin_a_low", ohlc.select(pl.col("coin_a_low")).to_series()
    ).to_list()[i]
    coin_a_high: float = pl.Series(
        "coin_a_high", ohlc.select(pl.col("coin_a_high")).to_series()
    ).to_list()[i]
    coin_a_close: float = pl.Series(
        "coin_a_high", ohlc.select(pl.col("coin_a_close")).to_series()
    ).to_list()[i]

    # actual buy -> give base, take target
    if order_type == "buy":
        # list of profits for executed orders
        inner_profits: list[float] = []
        inner_base_token_gains: list[float] = []

        for order_level in range(1, order_levels + 1):
            # get buy order level price
            buy_order_level: float = pl.Series(
                f"buy_level_{order_level}",
                ohlc.select(pl.col(f"buy_level_{order_level}")).to_series(),
            ).to_list()[i]

            # get hedge sell order level price for corresponding buy order level
            hedge_sell_order: float = pl.Series(
                f"hedge_sell_{order_level}",
                ohlc.select(pl.col(f"hedge_sell_{order_level}")).to_series(),
            ).to_list()[i]

            # check if buy order level is in range
            if coin_a_low < buy_order_level < coin_a_high:
                print(f"buy order level: {order_level} --> high: {coin_a_high}, order: {buy_order_level}, low: {coin_a_low}, hedge: {hedge_sell_order}")
                gains: float = calculate_tokens(initial_tokens, buy_order_level, hedge_sell_order, order_type)
                inner_base_token_gains.append(round(gains, round_off))
                inner_profits.append(round(gains, round_off))
            else:
                # no order executed
                print("no orders executed...")
                inner_base_token_gains.append(0)
                inner_profits.append(0)
            
        # calculate total ada gains
        base_token_gains.append(sum(inner_base_token_gains))
        profits_data.append(sum(inner_profits))
        print(f"inner_base_token_gains: {inner_base_token_gains}")
        print(f"inner_profits: {inner_profits}")
        
    # actual sell -> give target, take base
    elif order_type == "sell":
        # list of profits for executed orders
        inner_profits: list[float] = []
        inner_target_token_gains: list[float] = []

        for order_level in range(1, order_levels + 1):
            # get sell order level price
            sell_order_level: float = pl.Series(
                f"sell_level_{order_level}",
                ohlc.select(pl.col(f"sell_level_{order_level}")).to_series(),
            ).to_list()[i]

            # get hedge buy order level price for corresponding sell order level
            hedge_buy_order: float = pl.Series(
                f"hedge_buy_{order_level}",
                ohlc.select(pl.col(f"hedge_buy_{order_level}")).to_series(),
            ).to_list()[i]

            # check if sell order level is in range
            if coin_a_low < sell_order_level < coin_a_high:
                print(f"buy order level: {order_level} --> high: {coin_a_high}, order: {sell_order_level}, low: {coin_a_low}, hedge: {hedge_buy_order}")
                gains: float = calculate_tokens(initial_tokens, sell_order_level, hedge_buy_order, order_type)
                inner_target_token_gains.append(round(gains, round_off))
                inner_profits.append(round(gains*coin_a_close, round_off))
            else:
                # no order is executed
                print("no orders executed...")
                inner_target_token_gains.append(0)
                inner_profits.append(0)
                
        # calculate total actual gains
        target_token_gains.append(sum(inner_target_token_gains))
        profits_data.append(sum(inner_profits))
        print(f"inner_target_token_gains: {inner_target_token_gains}")
        print(f"inner_profits: {inner_profits}")
    
    print(f"total_profits: {round(sum(profits_data), round_off)}, base_token_gains: {round(sum(base_token_gains), round_off)}, target_token_gains: {round(sum(target_token_gains), round_off)}")
    return round(sum(profits_data), round_off), round(sum(base_token_gains), round_off), round(sum(target_token_gains), round_off)

In [ ]:
# calculate profits

profits: list[float] = []
base_tokens: int = 100
target_tokens: int = 222
base_token_gains: list[float] = []
target_token_gains: list[float] = []

for i in range(ohlc.shape[0]):
    # get buy and sell orders executed from ohlc
    buy_order_executed: int = (
        ohlc.select(pl.col("buy_order_executed")).to_series().to_list()[i]
    )
    sell_order_executed: int = (
        ohlc.select(pl.col("sell_order_executed")).to_series().to_list()[i]
    )
    inner_profits: list[float] = []
    inner_base_token_gains: list[float] = []
    inner_target_token_gains: list[float] = []

    print(
        f"------------------------------------------{i}th row in ohlc dataFrame----------------------------------------------"
    )

    if buy_order_executed > 0:
        print(
            "-----------------------------------------------buy order--------------------------------------------------------"
        )
        row_profits, row_base_token_gains, row_target_token_gains = calculate_profits(i, "buy", ohlc, base_tokens)
        inner_profits.append(row_profits)
        inner_base_token_gains.append(row_base_token_gains)
        inner_target_token_gains.append(row_target_token_gains)
    else:
        print(
            "----------------------------------------------no buy order------------------------------------------------------"
        )

    if sell_order_executed > 0:
        print(
            "-----------------------------------------------sell order-------------------------------------------------------"
        )
        row_profits, row_base_token_gains, row_target_token_gains = calculate_profits(i, "sell", ohlc, target_tokens)
        inner_profits.append(row_profits)
        inner_base_token_gains.append(row_base_token_gains)
        inner_target_token_gains.append(row_target_token_gains)
    else:
        print(
            "----------------------------------------------no sell order-----------------------------------------------------"
        )

    profits.append(sum(inner_profits))
    base_token_gains.append(sum(inner_base_token_gains))
    target_token_gains.append(sum(inner_target_token_gains))

ohlc = ohlc.insert_column(column=pl.Series("profits", profits), index=ohlc.shape[1])
ohlc = ohlc.insert_column(column=pl.Series("ada_token_gains", base_token_gains), index=ohlc.shape[1])
ohlc = ohlc.insert_column(column=pl.Series("actual_token_gains", target_token_gains), index=ohlc.shape[1])

ohlc.head(20)

In [ ]:
# ohlc profits sum

print("ohlc profits sum: ", ohlc.select(pl.col("profits")).sum())
print("ohlc bid order executed sum: ", ohlc.select(pl.col("buy_order_executed")).sum())
print("ohlc ask order executed sum: ", ohlc.select(pl.col("sell_order_executed")).sum())

# # ohlc flat fees
# ohlc = ohlc.with_columns(
#     ((pl.col("buy_order_executed") + pl.col("sell_order_executed")) * 2).alias(
#         "flat_fees"
#     )
# )
# print("ohlc flat fees sum: ", ohlc.select(pl.col("flat_fees")).sum())

# ohlc.head(5)

# # # net profit
# net_profit = (
#     ohlc.select(pl.col("profits")).sum() - ohlc.select(pl.col("flat_fees")).sum()
# )

# print("Net Profits: ", net_profit)

# print("base_token_gains: ", ohlc.select(pl.col("base_token_gains")).sum())
# print("target_token_gains: ", ohlc.select(pl.col("target_token_gains")).sum())